# **ETL AUSTRALIAN_USERS_REVIEWS_ANALYSIS**

## 1. Importación de Librerías

In [1]:
%pip install emoji --upgrade

   ---------------------------------------- 0.0/421.5 kB ? eta -:--:--
    --------------------------------------- 10.2/421.5 kB ? eta -:--:--
   -- ------------------------------------ 30.7/421.5 kB 435.7 kB/s eta 0:00:01
   ----- --------------------------------- 61.4/421.5 kB 544.7 kB/s eta 0:00:01
   ---------------------- ----------------- 235.5/421.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 421.5/421.5 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install nltk

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 406.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.5 MB 1.6 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.5 MB 2.9 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 4.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/268.9 kB ? eta -:--:--
   --------------------------------------- 268.9/268.9 kB 16.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import numpy as np # Cargamos la librería de "numpy" para realizar cálculos lógicos y matemáticos sobre cuadros y matrices en el caso que lo necesitemos
import chardet #Cargamos la librería "chardet" que nos permitirá saber el tipo de enconding que tiene nuestro archivo ha analizar
import json #Cargando la librería "json" nos permitira manipular archivos tipo JSON
import ast #Cargaremos ast para poder extraer datos de json
import datetime #Cargaremos datetime para poder extraer la fecha de las columnas
import re , emoji #Cargaremos re y emoji para poder usar expresiones regulares
import nltk #Cargaremos nltk para poder usar expresiones regulares
#Recursos necesarios para el procesamiento de texto
nltk.download('stopwords')
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CRISTHIAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CRISTHIAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. ETL (Extract - Transform - Load)

### 2.1. Carga de DataSet

In [5]:
#Colocamos la ruta del archivo JSON
user_reviews_ruta = r'Dataset\australian_user_reviews.json'

In [6]:
#Utilizamos la liberia "chardet" para saber la codificación del archivo
with open(user_reviews_ruta, 'rb') as archivo:
    resultado = chardet.detect(archivo.read())

# Imprimimos la codificación detectada
print(f"La codificación detectada es: {resultado['encoding']}")

La codificación detectada es: MacRoman


In [7]:
#Leemos el archivo JSON lo cargamos con el encoding detectado y lo guardamos a una lista "filas_reviews"
filas_reviews=[]
with open(user_reviews_ruta, encoding="MacRoman") as file:
    for line in file.readlines():
        filas_reviews.append(ast.literal_eval(line))

# Se convierte en dataframe
df_user_reviews = pd.DataFrame(filas_reviews)

In [8]:
#Visualizamos los primeras 5 filas del archivo cargado en un DataFrame
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### 2.2. Analisis Descriptivo del DataSet

#### 2.2.1. Definición de las Funciones Descriptivas del DataSet

In [9]:
#Definimos algunas funciones para que nos facilita la descripcion de las principales caracteristicas del DataFrame
def caracteristicas_df(df):
    """
    Describe de forma general la base de datos .

    Esta función simplemente muestra el tamaño, información general y
    la cantidad de datos nulos.

    Parametros
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        - 'df.shape': Numero de filas y columnas
        - 'df.info': Muestra información general del DataFrame

    """
    print('*'*10 + '|'*10 + 'FORMA DE BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(f'Tiene {df.shape[0]} filas y {df.shape[1]} columnas o variables')
    print(end = '\n'*2)

    print('*'*10 + '|'*10 + 'INFORMACION GENERAL DE LA BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(df.info(), end = '\n'*2)

def valores_nulos_df(df):
    """
    Revisa presencia de valores nulos en un DataFrame.
    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    el porcentaje de valores no nulos y nulos, así como la ncantidad de valores nulos por columna.

    Parametros:
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre': Nombre de cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.

    """
    mi_df = {"nombre": [], "tipo_datos": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_df["nombre"].append(columna)
        mi_df["tipo_datos"].append(df[columna].apply(type).unique())
        mi_df["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_df["nulos"].append(df[columna].isnull().sum())

    df_nulos = pd.DataFrame(mi_df)

    return df_nulos

#### 2.2.2. Descripcion del DataSet

In [10]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 25799 filas y 3 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB
None



In [11]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,reviews,[<class 'list'>],0.0,0


#### 2.2.3. Revisamos algunos datos de columnas para conocer su estructura:

In [12]:
#Seleccionamos un datos aleatorio en la columna 'user_id' para ver el contenido
df_user_reviews['user_id'][20]

'Fr0stedLine'

In [13]:
#Seleccionamos un datos aleatorio en la columna 'user_url' para ver el contenido
df_user_reviews['user_url'][55]

'http://steamcommunity.com/id/Life-Of-Blu'

In [14]:
#Seleccionamos un datos aleatorio en la columna 'reviews' para ver el contenido
df_user_reviews['reviews'][4]

[{'funny': '3 people found this review funny',
  'posted': 'Posted April 15, 2014.',
  'last_edited': '',
  'item_id': '211420',
  'helpful': '35 of 43 people (81%) found this review helpful',
  'recommend': True,
  'review': 'Git gud'},
 {'funny': '1 person found this review funny',
  'posted': 'Posted December 23, 2013.',
  'last_edited': '',
  'item_id': '211820',
  'helpful': '12 of 16 people (75%) found this review helpful',
  'recommend': True,
  'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
 {'funny': '2 people found this review funny',
  'posted': 'Posted March 14, 2014.',
  'last_edited': '',
  'item_id': '730',
  'helpful': '5 of 5 people (100%) found this review helpful',
  'recommend': True,
  'review': 'Hold shift to win, Hold CTRL to lose.'},
 {'funny': '',
  'posted': 'Posted July 11, 2013.',
  'last_edited': '',
  'item_id': '204300',
  'helpful': 'No ratings yet',
  'recommend': True,
  'rev

Este conjunto contiene 3 columnas y 25799 filas, sin valores nulos. Las columnas son:

*   **user_id:** es un identificador único para el usuario.
*   **user_url:** es la url del perfil del usuario en streamcommunity.
*   **reviews:** contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionario con el review. Cada diccionario contiene:
    *   **funny:** indica si alguien puso emoticón de gracioso al review.
    *   **posted:** es la fecha de posteo del review en formato Posted April 21, 2011.
    *   **last_edited:** es la fecha de la última edición.
    *   **item_id:** es el identificador único del item, es decir, del juego.
    *   **helpful:** es la estadística donde otros usuarios indican si fue útil la información.
    *   **recommend:** es un booleano que indica si el usuario recomienda o no el juego.
    *   **review:** es una sentencia string con los comentarios sobre el juego.

### 2.3. Transformación de Datos

#### 2.3.1. Funciones de Transformación

In [35]:
def verifica_duplicados(df, columna):
    '''
    Verifica y muestra filas duplicadas en un DataFrame basado en una columna específica.

    Esta función toma como entrada un DataFrame y el nombre de una columna específica.
    Luego, identifica las filas duplicadas basadas en el contenido de la columna especificada,
    las filtra y las ordena para una comparación más sencilla.

    Parameters:
    ----------
        df (pandas.DataFrame): El DataFrame en el que se buscarán filas duplicadas.
        columna (str): El nombre de la columna basada en la cual se verificarán las duplicaciones.

    Returns:
    ----------
        pandas.DataFrame or str: Un DataFrame que contiene las filas duplicadas filtradas y ordenadas,
        listas para su inspección y comparación, o el mensaje "No hay duplicados" si no se encuentran duplicados.
    '''
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"

    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted



def date_post(text):

  """
  Extrae la fecha de la columna "posted" donde podemos extraer 2 formatos diferentes 1 de ellos "%B %d, %Y y el otro un formato "%B %d haciendo falta %Y.

  Es una función que procesa y convierte una cadena
  de texto que representa una fecha en un objeto "datetime.datetime".

  Parameters:
  ----------
  text: El valor que de cual se extraera la fecha de posteo del review.

  Returns:
  ----------
  date_str: Fecha con el formato deseado.

  """

  # Eliminar espacios en blanco al principio y al final de la cadena
  date_string = text.strip().lstrip('Posted').rstrip('.')

  if date_string.count(',') > 0:
      date_str = datetime.datetime.strptime(date_string, " %B %d, %Y").date()
  else:
      # Después de un análisis de los datos de la columna date post se concluyó que nuestra información se encuentra entre los años 2010-2015 y una parte de estas fechas no tienen año.
      # La mayoría son en febrero, por lo tanto, se procesará como si fuera el año 2012 ya que hay una probabilidad de que se hayan confundido al digitar el año con el mes
      año_bisiesto = 2012
      date_string = date_string.replace(',', '') + f', {año_bisiesto}'
      date_str = datetime.datetime.strptime(date_string, " %B %d, %Y").date()

  return date_str


def date_edit(text):

  """
  Esta función toma como entrada un valor tipo texto o str.
  Extrae la fecha de la columna "last_edited" donde podemos extraer 2 formatos diferentes:
    - 1. "%B %d, %Y donde: %B representa el nombre completo del mes, %d, representa el día del mes como un número decimal rellenado con ceros
    y %Y representa el año con el siglo como un número decimal.
    - 2. El otro formato es para fechas vacias o tipo NaN.

  Parameters:
  ----------
  text: El valor que de cual extraera la fecha de la columna "date_edit.

  Returns:
  ----------
  date_str: Fecha con el formato deseado en este caso un str.

  """

  # Eliminar espacios en blanco al principio y al final de la cadena
  date_string = text.lstrip('Last edited ').strip()

  # Eliminar cualquier carácter no deseado al final de la cadena
  date_string = date_string.rstrip('.')

  if date_string and date_string.count(',') > 0:
      try:
          date_str = datetime.datetime.strptime(date_string, "%B %d, %Y")
      except ValueError:
          # Si hay un error, asumir que es NaN o un formato no reconocido
          date_str = np.nan
  else:
      date_str = np.nan

  return date_str


def extract_funny_count(text):

  """
  Procesa una cadena que indica la cantidad de personas que les parecio divertido el juego en la columna "reviews"

  Esta función toma como entrada un valor tipo texto o str. Luego se elimina la parte final de la cadena que indica cuántas personas
  encontraron divertido el juego. Primero, se quita la subcadena 'person found this review funny', y luego, si aún queda,
  se elimina ' people found this review funny'. Además, se quitan las comas de la cadena.

  Parameters:
  ----------
  text: El valor que de cual se contara la cantidad de personas.

  Returns:
  ----------
  funny_count: Conteo total de cuantas personas indicaron que fue divertido el juego.

  """

  funny_count=text.rstrip(' person found this review funny').rstrip(' people found this review funny').replace(',','')

  if len(funny_count)>0:
    funny_count =  int(funny_count)
  else:
    funny_count =  0
  return funny_count


def extract_helpful_count(text):

  """
  Procesa una cadena que indica la cantidad de votos que califican como util o de ayuda la critica realizada

  Esta función toma como entrada un valor tipo texto o str, el código procesa la cadena text para extraer información sobre votos o calificaciones.
  Si la cadena es 'No ratings yet', se devuelve [0, 0]. Si la cadena contiene información sobre votos, se extraen y convierten a enteros, y se devuelve solo el primer valor.

  Parameters:
  ----------
  text: El valor que de cual se contara la cantidad de votos.

  Returns:
  ----------
  int(y[0]): Cantidad de votos

  """

  part1, part2 = 0, 0

  if text == 'No ratings yet':
      y=[0,0]
  else:
      vote_string = text[:text.index("p")].replace(' ', '').replace('of', ',')
      y = vote_string.split(',')
  #return [int(y[0]), int(y[1])]
  return int(y[0])



def extract_total_count(text):

  """
  Procesa una cadena que indica la cantidad de votos en total realizadas por critica.

  Esta función toma como entrada un valor tipo texto o str, el código procesa la cadena text para extraer información sobre votos o calificaciones.
  Si la cadena es 'No ratings yet', se devuelve [0, 0]. Si la cadena contiene información sobre votos, se extraen y convierten a enteros, y se devuelve solo el primer valor.

  Parameters:
  ----------
  text: El valor que de cual se contara la cantidad de votos.

  Returns:
  ----------
  int(y[0]): Cantidad de votos

  """

  part1, part2 = 0, 0

  if text == 'No ratings yet':
      y=[0,0]
  else:
      vote_string = text[:text.index("p")].replace(' ', '').replace('of', ',')
      y = vote_string.split(',')
  return int(y[1])



def count_emojis(x):

  ''''
  Procesa una cadena para contar los emojis que se encontraban en la columna "review"

  Esta función toma como entrada un valor tipo texto o str, el código procesa la cadena text para utilizar la biblioteca emoji para encontrar y contar los emojis en una cadena.
  Devuelve una lista con los emojis encontrados o una cadena vacía si no se encuentran emojis.

  Parameters:
  ----------
  x: El valor del cual se contara los emojis

  Returns:
  ----------
  emojis_count: Cantidad de emojis

  '''

  emojis_count = []
  emojis_encontrados  = emoji.emoji_list(x)
  if emojis_encontrados == '':
      return ''
  for e in emojis_encontrados:
      emojis_count.append(e['emoji'])
  return emojis_count



def unique_emojis(emojis):

  '''
  Procesa una lista para eliminar los duplicados y devuelve con valores unicos.

  Esta función toma como entrada un valor tipo lista, este código elimina los duplicados de la lista emojis utilizando un conjunto y
  luego devuelve una nueva lista con los emojis únicos preservando el orden original.

  Parameters:
  ----------
  emojis: lista de emojis

  Returns:
  ----------
  list(set(emojis)): Lista de emojis.

  '''
  return list(set(emojis))


def emojis_to_words(text):

  '''
  Procesa str u emoji y convertirlo a texto original o palabra reconocible.

  Esta función toma como entrada un valor tipo texto o str, luego itera sobre un diccionario que mapea emojis a sus reemplazos
  y reemplazar cada instancia de emoji en la cadena de texto original con su correspondiente valor de reemplazo.

  Parameters:
  ----------
  text: lista de emojis

  Returns:
  ----------
  text: Lista de emojis.

  '''

  for emoji, replacement in emoji_reemplazar.items():
      text = text.replace(emoji, replacement)
  return text

# Llamando a las stopwords en inglés
stopwords = nltk.corpus.stopwords.words('english')
stopwords = [word for word in stopwords if word != 'not']


def delete_stopwords(x):
  """
  Elimina las palabras vacías (stopwords) de una cadena de texto.

  Parameters:
  ----------
  x: Cadena de texto de entrada.

  Returns:
  ----------
  str: Cadena de texto sin palabras vacías.
  """

  # Tokeniza la cadena de texto y elimina las stopwords
  palabras_filtradas = [palabra for palabra in nltk.tokenize.word_tokenize(x) if palabra not in stopwords]

  # Une las palabras filtradas en una sola cadena de texto
  texto_sin_stopwords = " ".join(palabras_filtradas)

  return texto_sin_stopwords


def clean_text(x):
  '''
  Limpia una cadena de texto eliminando caracteres especiales y dejando solo letras y espacios.

  Parameters:
  ----------
  x: Cadena de texto de entrada.

  Returns:
  ----------
  str: Cadena de texto limpia.
  '''

  # Elimina caracteres no alfabéticos y deja solo letras y espacios
  texto_limpio = re.sub(r'[^a-zA-Z\s]', '', x)

  return texto_limpio

#### 2.3.2. Transformación y eliminación de datos de TODO el DataFrame

In [16]:
#Verificamos si existen duplicados en la columna "user_id", si existen eliminar
filas_duplicadas = verifica_duplicados(df_user_reviews, 'user_id')

#Revisamos la fila de reviews si en verdad se repiten y SI se repiten
filas_duplicadas['reviews']

12888    [{'funny': '', 'posted': 'Posted May 18, 2015....
5250     [{'funny': '', 'posted': 'Posted May 18, 2015....
3133     [{'funny': '', 'posted': 'Posted December 22, ...
3134     [{'funny': '', 'posted': 'Posted December 22, ...
4139     [{'funny': '', 'posted': 'Posted March 26.', '...
                               ...                        
2721     [{'funny': '', 'posted': 'Posted July 17, 2015...
2680     [{'funny': '', 'posted': 'Posted October 31, 2...
17916    [{'funny': '', 'posted': 'Posted October 31, 2...
5855     [{'funny': '', 'posted': 'Posted November 30, ...
13975    [{'funny': '', 'posted': 'Posted November 30, ...
Name: reviews, Length: 623, dtype: object

In [17]:
#Eliminamos los valores duplicados
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')

#Verificamos si aún
verifica_duplicados(df_user_reviews, 'user_id')

'No hay duplicados'

#### 2.3.3. Transformación y eliminación de datos en la columna "Reviews"

In [18]:
#Procedemos a explotar la columna "reviews" ya que tiene una lista de diccionarios
df_user_reviews =  df_user_reviews.explode('reviews')
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


In [19]:
#Verificamos si hay valores nulos en nuestro DataFrame luego de hacer la explosión
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.00,0
1,user_url,[<class 'str'>],0.00,0
2,reviews,"[<class 'dict'>, <class 'float'>]",0.05,28


In [20]:
#Observamos que tenemos 28 datos con valores nulos, tenemos que eliminar dicha data
df_user_reviews.dropna(subset = 'reviews', inplace=True)

#Verificamos si hay valores nulos
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,reviews,[<class 'dict'>],0.0,0


In [21]:
df_user_reviews['reviews'][0].to_list()

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [22]:
#Creamos una mascara donde vamos a guardar los datos que extramos de nuestra columna "reviews" del DataFrame Original
df = pd.DataFrame(columns=df_user_reviews['reviews'].iloc[0].keys())

In [23]:
#Lista vacia donde vamos a guardar los datos extraidos de la columna "reviews"
filas_nuevas = []

#Iteramos sobre los diccionarios en la columna 'reviews' del DataFrame original
for elemento, diccionario in df_user_reviews['reviews'].items():
    new_row = pd.DataFrame(diccionario, index=[elemento])  # Crear un nuevo DataFrame para una fila
    filas_nuevas.append(new_row)  # Agrega una fila nueva en la lista "filas_nuevas"

# Concatenar la lista de nuevas filas con el DataFrame original
df_resultado = pd.concat([df, *filas_nuevas])

# Reiniciar el índice
df_user_reviews = df_user_reviews.join(df_resultado)

# Eliminar duplicados basados en 'user_id' y 'posted' luego de la concatenacion
df_user_reviews.drop_duplicates(subset=['user_id', 'posted'], inplace=True)

In [24]:
#Eliminamos la columna "reviews" debido a que ya la desanidamos
df_user_reviews.drop(columns=['reviews'],inplace=True)

In [25]:
#Visualizamos las caracteristicas de nuestro nuevo DataFrame
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 54456 filas y 9 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
Index: 54456 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      54456 non-null  object
 1   user_url     54456 non-null  object
 2   funny        54456 non-null  object
 3   posted       54456 non-null  object
 4   last_edited  54456 non-null  object
 5   item_id      54456 non-null  object
 6   helpful      54456 non-null  object
 7   recommend    54456 non-null  object
 8   review       54456 non-null  object
dtypes: object(9)
memory usage: 4.2+ MB
None



In [26]:
#Verificamos si tiene datos nulos
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,funny,[<class 'str'>],0.0,0
3,posted,[<class 'str'>],0.0,0
4,last_edited,[<class 'str'>],0.0,0
5,item_id,[<class 'str'>],0.0,0
6,helpful,[<class 'str'>],0.0,0
7,recommend,[<class 'bool'>],0.0,0
8,review,[<class 'str'>],0.0,0


In [27]:
#Visualizamos nuestro nuevo DataFrame
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


#### 2.3.4. Transformación de las nuevas columnas

In [28]:
#Aplicamos las diferentes funciones que hemos creado en el punto 2.3.1. según corresponda:
df_user_reviews['Posted Date']  = df_user_reviews['posted'].apply(date_post) #Extraemos la fecha de la publicacion
df_user_reviews['Date last edited'] =  df_user_reviews['last_edited'].apply(date_edit) #Extraemos la fecha de la ultima edicion
df_user_reviews['funny review votes'] = df_user_reviews['funny'].apply(extract_funny_count) #Extraemos la cantidad de votos que calificaron como divertido
df_user_reviews['Helpful review votes'] = df_user_reviews['helpful'].apply(extract_helpful_count) #Extraemos la cantidad de votos que calificaron
df_user_reviews['total review votes'] = df_user_reviews['helpful'].apply(extract_total_count) #Extraemos la cantidad de votos totales
df_user_reviews['Date last edited']=df_user_reviews['Date last edited'].fillna(df_user_reviews['Posted Date']) #Reemplazamos los NaN por la fecha de publicacion
df_user_reviews.drop(columns=['posted','last_edited','funny','helpful'],inplace=True) #Eliminamos las columnas que no usaremos
df_user_reviews['recommend'] = df_user_reviews['recommend'].apply(lambda x: 1 if x == True else 0)

In [29]:
#Visualizamos nuestro DataFrame si se han aplicado las funciones de transformación
df_user_reviews.head()

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,Simple yet with great replayability. In my opi...,2011-11-05,2011-11-05,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,It's unique and worth a playthrough.,2011-07-15,2011-07-15,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011-04-21,0,0,0
1,js41637,http://steamcommunity.com/id/js41637,251610,1,I know what you think when you see this title ...,2014-06-24,2014-06-24,0,15,20
1,js41637,http://steamcommunity.com/id/js41637,227300,1,For a simple (it's actually not all that simpl...,2013-09-08,2013-09-08,0,0,1


In [30]:
#Revisamos la columnas "review" en el caso se nos haya escapado algo
df_user_reviews['review'].unique()

array(['Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.',
       "It's unique and worth a playthrough.",
       'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!',
       ...,
       'this game is a perfect remake of the original half life. personally one of the best remakes i have played in a long time. there are a few changes in the remake but for the most part its almost the same as the original half life.the game still needs Xen to be completed but all the other chapters are ready for you to play and enjoy. i say buy this game if you loved the original half life. but avoid it if you can t wait for xen to be completed.',
       'had so much fun plaing t

In [31]:
#Al revisar la columna reviews podemos observar que hay varios emojis " xD, :D, etc" en cada review por tanto procederemos a transformarlos a texto, los vamos a contar
emojis = df_user_reviews['review'].apply(lambda x: count_emojis(x))
emojis = emojis[emojis.apply(lambda x: len(x) > 0)]
emojis = emojis.apply(unique_emojis)
emojis.drop_duplicates(inplace=True)
emojis.count()

10

In [32]:
#Verificaremos que emojis son unicos o raros
emojis_unicos = []
for emojis_lista in emojis:
    emojis_unicos.extend(emojis_lista) # Recorremos la lista y los representamos

# Convertir la lista extendida en un conjunto para eliminar duplicados y luego en una lista nuevamente
emojis_unicos = list(set(emojis_unicos))
emojis_unicos

['©', '™', '🏻', '®']

In [33]:
#Vamos a crear un diccionario con los emojis más comunes en la columna "review" para luego reemplazarlas por texto
emoji_reemplazar = {
    '✔': 'Approved ',
    '💯': 'Perfect ',
    '♀': 'Woman ',
    '❤': 'Love ',
    '💋': 'Kiss ',
    '💀': 'Skull ',
    '™': 'Trademark ',
    '®': 'Registered Trademark ',
    '©': 'Copyright ',
    '☺': 'Smile ',
    '♥': 'Heart ',
    '👌': 'Okay ',
    '👀': 'Eye ',
    '⌛': 'Hourglass ',
    '☹': 'Sad ',
    '☑': 'Checked ',
    '😊': 'Happiness ',
    '😄': 'Happiness ',
    '◀': 'Left ',
    ':D': 'laugh ',
    ':3': 'smile',
    'xD': 'laughloud',
    'ô•': 'oh',
    ':c': 'adictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-' : 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'adictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'
}
#Aplicamos la transformación
df_user_reviews['review'] = df_user_reviews['review'].apply(emojis_to_words)
#Pasamos a minusculas los textos dentro de la columna "review"
df_user_reviews['review'] = df_user_reviews['review'].str.lower()
#Visualizamos si efectivamente se hizo la transformación
df_user_reviews.head()

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,simple yet with great replayability. in my opi...,2011-11-05,2011-11-05,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,it's unique and worth a playthrough.,2011-07-15,2011-07-15,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,1,great atmosphere. the gunplay can be a bit chu...,2011-04-21,2011-04-21,0,0,0
1,js41637,http://steamcommunity.com/id/js41637,251610,1,i know what you think when you see this title ...,2014-06-24,2014-06-24,0,15,20
1,js41637,http://steamcommunity.com/id/js41637,227300,1,for a simple (it's actually not all that simpl...,2013-09-08,2013-09-08,0,0,1


#### 2.3.5. Transformación y Preprocesamiento de Texto  

In [36]:
#Aplicamos las diferentes funciones que hemos creado en el punto 2.3.1. según corresponda:
df_user_reviews['review'] = df_user_reviews['review'].apply(delete_stopwords)
df_user_reviews['review'] = df_user_reviews['review'].apply(clean_text)
df_user_reviews.reset_index(drop=True, inplace=True)

In [37]:
#Visualizamos las caracteristicas de nuestro nuevo DataFrame
caracteristicas_df(df_user_reviews)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 54456 filas y 10 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54456 entries, 0 to 54455
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   user_id               54456 non-null  object        
 1   user_url              54456 non-null  object        
 2   item_id               54456 non-null  object        
 3   recommend             54456 non-null  int64         
 4   review                54456 non-null  object        
 5   Posted Date           54456 non-null  object        
 6   Date last edited      54456 non-null  datetime64[ns]
 7   funny review votes    54456 non-null  int64         
 8   Helpful review votes  54456 non-null  int64         
 9   total review votes    54456 non-null  int64         
dtyp

In [38]:
#Verificamos si tiene datos nulos
valores_nulos_df(df_user_reviews)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,user_url,[<class 'str'>],0.0,0
2,item_id,[<class 'str'>],0.0,0
3,recommend,[<class 'int'>],0.0,0
4,review,[<class 'str'>],0.0,0
5,Posted Date,[<class 'datetime.date'>],0.0,0
6,Date last edited,[<class 'pandas._libs.tslibs.timestamps.Timest...,0.0,0
7,funny review votes,[<class 'int'>],0.0,0
8,Helpful review votes,[<class 'int'>],0.0,0
9,total review votes,[<class 'int'>],0.0,0


In [39]:
#Para evitar complicaciones al momento de exportar el DataSet vamos a buscar si existen datos que se podrian convertir en datos nulos
df_user_reviews[df_user_reviews['review'] == '']

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
81,boydeer,http://steamcommunity.com/id/boydeer,383080,0,,2015-08-24,2015-08-24,0,4,7
301,76561198078478151,http://steamcommunity.com/profiles/76561198078...,730,1,,2012-04-10,2012-04-10,0,0,0
476,76561198084542731,http://steamcommunity.com/profiles/76561198084...,206420,1,,2015-01-16,2015-01-16,0,0,0
578,76561198070263209,http://steamcommunity.com/profiles/76561198070...,570,1,,2013-12-13,2013-12-13,0,0,0
809,Aurora99,http://steamcommunity.com/id/Aurora99,282440,0,,2015-02-15,2015-02-15,1,0,2
...,...,...,...,...,...,...,...,...,...,...
54078,42456,http://steamcommunity.com/id/42456,304930,1,,2014-07-30,2014-07-30,2,2,4
54156,SoraThailand,http://steamcommunity.com/id/SoraThailand,440,1,,2014-09-15,2014-09-15,0,0,0
54344,huangsy,http://steamcommunity.com/id/huangsy,304930,1,,2012-06-11,2012-06-11,1,6,9
54372,76561198209894493,http://steamcommunity.com/profiles/76561198209...,387990,1,,2012-02-24,2012-02-24,0,0,1


In [40]:
#Pasaremos a eliminar estas filas con el valor "" que puede interferir en el caso de la exportación
df_user_reviews = df_user_reviews.drop(df_user_reviews[df_user_reviews['review'] == ''].index)

In [41]:
#Volvemos a visualizar las caracteristicas
caracteristicas_df(df_user_reviews) #Ahora tendremos 53902 filas

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 53902 filas y 10 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
Index: 53902 entries, 0 to 54455
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   user_id               53902 non-null  object        
 1   user_url              53902 non-null  object        
 2   item_id               53902 non-null  object        
 3   recommend             53902 non-null  int64         
 4   review                53902 non-null  object        
 5   Posted Date           53902 non-null  object        
 6   Date last edited      53902 non-null  datetime64[ns]
 7   funny review votes    53902 non-null  int64         
 8   Helpful review votes  53902 non-null  int64         
 9   total review votes    53902 non-null  int64         
dtypes: d

In [42]:
#Visualizamos nuestro DataFrame Final
df_user_reviews.head()

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,simple yet great replayability opinion zombi...,2011-11-05,2011-11-05,0,0,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,s unique worth playthrough,2011-07-15,2011-07-15,0,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,1,great atmosphere gunplay bit chunky times end...,2011-04-21,2011-04-21,0,0,0
3,js41637,http://steamcommunity.com/id/js41637,251610,1,know think see title barbie dreamhouse party ...,2014-06-24,2014-06-24,0,15,20
4,js41637,http://steamcommunity.com/id/js41637,227300,1,simple s actually not simple truck driving ...,2013-09-08,2013-09-08,0,0,1


### 2.4. Exportación de DataSet Limpia

In [45]:
#Se guarda el dataframe transformado como australian_user_reviews_clean
archivo_limpio = r'Dataset_Clean\australian_user_reviews_clean.csv'
df_user_reviews.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Dataset_Clean\australian_user_reviews_clean.csv


In [46]:
#Se guarda el dataframe transformado como australian_user_reviews_clean
archivo_limpio_csv = r'Dataset_Clean\australian_user_reviews_clean.csv'
archivo_limpio_parquet = r'Dataset_Clean\australian_user_reviews_clean.parquet'
# Leemos el archivo CSV en un DataFrame
df = pd.read_csv(archivo_limpio_csv)
df.to_parquet(archivo_limpio_parquet, engine='pyarrow')
print(f'Se guardó el archivo {archivo_limpio_parquet}')

Se guardó el archivo Dataset_Clean\australian_user_reviews_clean.parquet
